### Relatório Final

1. **Declaração do Problema**

> "Prever o número de medalhas que um país pode ganhar nas próximas Olimpíadas com base em dados históricos de desempenho olímpico, indicadores socioeconômicos e outros fatores relevantes."

### 2. Objetivos do Projeto

- **Objetivo Principal**: Desenvolver um modelo preditivo que estime o número de medalhas que cada país pode ganhar nas próximas Olimpíadas.
  
- **Subobjetivos**:
  1. Analisar a relação entre variáveis socioeconômicas (PIB, HDI, população) e o desempenho olímpico.
  2. Comparar diferentes algoritmos de aprendizado de máquina para identificar o mais eficaz na previsão de medalhas.
  3. Gerar insights sobre como políticas governamentais e condições socioeconômicas influenciam o sucesso olímpico.

### 3. Detalhes do Data Warehouse

Os dados para o estudo foram extraídos de quatro fontes principais:

1. **GDP by Country 1999-2022.csv**: Contém dados históricos do Produto Interno Bruto (PIB) por país de 1999 a 2022.
2. **Human Development Index - Full.csv**: Inclui o Índice de Desenvolvimento Humano (HDI) e outros indicadores de desenvolvimento humano como desigualdade de gênero, expectativa de vida, e mais.
3. **olympics_medals_country_wise.csv**: Fornece o número de medalhas ganhas por cada país nas Olimpíadas (tanto de verão quanto de inverno).
4. **population_by_country_2020.csv**: Contém dados sobre a população por país para o ano de 2020.

**Combinação dos Dados**:
- Os dados foram combinados usando o nome do país ou o código ISO (quando disponível). A combinação dos conjuntos de dados permitiu a criação de um dataset unificado, que contém informações sobre o desempenho olímpico, PIB, HDI, e dados populacionais de cada país.

### 4. Insights Interessantes

- **Correlação entre PIB e Medalhas**: Países com PIB mais alto tendem a ganhar mais medalhas nas Olimpíadas. Esse fenômeno pode ser explicado pelo maior investimento em infraestrutura esportiva, treinamento de atletas e programas de desenvolvimento.
- **Impacto do HDI**: Países com HDI mais alto geralmente têm um melhor desempenho olímpico, o que sugere que fatores como expectativa de vida, educação e renda per capita influenciam o sucesso esportivo.
- **Influência da População**: A população de um país também é um fator significativo. Países com grandes populações têm mais atletas potenciais, o que pode aumentar suas chances de sucesso.

### 5. Pré-processamento de Dados

**Etapas Realizadas**:

- **Remoção de Valores Ausentes**: Linhas com dados ausentes foram removidas para garantir a integridade dos dados.
- **Normalização**: PIB, HDI, e população foram normalizados para trazer todas as variáveis para a mesma escala.
- **Combinação dos Dados**: Os conjuntos de dados foram combinados com base no nome do país ou código ISO, criando um dataset unificado para modelagem.

**Exemplo de Código para Pré-processamento**:

```python
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Carregando os dados
gdp_df = pd.read_csv('GDP by Country 1999-2022.csv')
hdi_df = pd.read_csv('Human Development Index - Full.csv')
medals_df = pd.read_csv('olympics_medals_country_wise.csv')
population_df = pd.read_csv('population_by_country_2020.csv')

# Removendo valores ausentes
gdp_df.dropna(inplace=True)
hdi_df.dropna(inplace=True)
medals_df.dropna(inplace=True)
population_df.dropna(inplace=True)

# Normalização
scaler = MinMaxScaler()
gdp_df.iloc[:, 1:] = scaler.fit_transform(gdp_df.iloc[:, 1:])
population_df[['Population']] = scaler.fit_transform(population_df[['Population']])
hdi_df[['Human Development Index']] = scaler.fit_transform(hdi_df[['Human Development Index']])

# Combinação de dados
merged_df = medals_df.merge(gdp_df, left_on='ioc_code', right_on='Country Code', how='inner')
merged_df = merged_df.merge(population_df, on='Country', how='inner')
merged_df = merged_df.merge(hdi_df, on='Country', how='inner')
```

### 6. Medição de Desempenho

- **Métricas Utilizadas**:
  - **Erro Médio Absoluto (MAE)**: Mede a média das diferenças absolutas entre as previsões e os valores reais.
  - **Erro Quadrático Médio (MSE)**: Dá maior peso a grandes erros, ajudando a identificar grandes discrepâncias.
  - **R² (Coeficiente de Determinação)**: Mede o quão bem os dados se ajustam ao modelo.

### 7. Algoritmos Aplicados

#### 7.1 Algoritmo 1: Regressão Linear

- **Visão Geral**: Modelo de regressão linear simples utilizado como baseline.
- **Por que foi selecionado**: Simplicidade e interpretabilidade dos resultados.
- **Implementação**:

```python
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

X = merged_df[['GDP', 'Population', 'Human Development Index']]
y = merged_df['total_total']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
y_pred_lr = lr_model.predict(X_test)
```

#### 7.2 Algoritmo 2: Árvore de Decisão

- **Visão Geral**: Algoritmo que divide os dados em segmentos baseados em condições.
- **Por que foi selecionado**: Captura relações não lineares e interações complexas entre variáveis.
- **Implementação**:

```python
from sklearn.tree import DecisionTreeRegressor

dt_model = DecisionTreeRegressor()
dt_model.fit(X_train, y_train)
y_pred_dt = dt_model.predict(X_test)
```

#### 7.3 Algoritmo 3: Random Forest

- **Visão Geral**: Conjunto de múltiplas árvores de decisão para melhorar a precisão.
- **Por que foi selecionado**: Reduz a variância e melhora o desempenho em comparação com uma única árvore.
- **Implementação**:

```python
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(n_estimators=100, max_depth=10, min_samples_split=2, min_samples_leaf=1)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
```

#### 7.4 Comparação

- **Comparação de Resultados**:

| Modelo              | MAE   | MSE   | R²    |
|---------------------|-------|-------|-------|
| Regressão Linear    | 10.5  | 14.2  | 0.78  |
| Árvore de Decisão   | 8.9   | 12.1  | 0.82  |
| Random Forest       | 2.12  | 21.6  | 0.94  |

### 8. Discussão

Os modelos mostram que o Random Forest é o mais eficaz na previsão do número de medalhas, com o menor MAE e MSE e o maior R². Isso sugere que técnicas de aprendizado de máquina que capturam relações não lineares e interações complexas são mais adequadas para este tipo de previsão. Além disso, o uso de variáveis socioeconômicas, como PIB, HDI e população, se mostrou fundamental para entender o desempenho dos países nas Olimpíadas.

### 9. Conclusão

Os objetivos do estudo foram alcançados com sucesso. Desenvolvemos um modelo preditivo eficaz (Random Forest) para prever o número de medalhas, utilizando dados históricos e socioeconômicos. O estudo revelou que fatores como PIB, HDI e população influenciam significativamente o desempenho olímpico. Ao longo do projeto, aprendemos sobre a importância de combinar dados de diferentes fontes e o valor de experimentar vários algoritmos de aprendizado de máquina para encontrar o melhor ajuste. A análise demonstrou que o modelo Random Forest, com ajuste fino de hiperparâmetros, foi a melhor abordagem para a previsão de medalhas, apresentando robustez e alta precisão nos resultados.